## Packages

In [15]:
import pandas as pd
import re
import string
import nltk
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

## Functions

In [16]:
def transformDocument(df, column_name, language):
    stop_words = usingStopwords(language)
    df_pp = df.copy()
    # 1. Aplicar preprocessamento nos títulos e textos completos
    if language == 'pt':
        # Substituir símbolos importantes
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-feira', ''))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('+', 'mais '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-', 'menos '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('%', ' por cento'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: removeStopwords(s, stop_words))

    elif language == 'en':
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-', 'less'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('+', 'plus '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('%', ' percent'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: removeStopwords(s, stop_words))

    else:
        pass

    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('R$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('U$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('US$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('S&P 500', 'spx'))

    # Transformar em String e Letras Minúsculas nas Mensagens
    df_pp[column_name] = df_pp[column_name].map(lambda s:
                                              normalizarString(s))

    # Remover Pontuações
    # Remover Pontuações
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis
    df_pp[column_name] = df_pp[column_name].map(lambda s: removeEmojis(s))

    # Quebras de Linha desnecessárias
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('\"', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('“', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('”', ''))

    # Remover valores
    df_pp[column_name] = df_pp[column_name].map(lambda s: removeValores(s))

    # Espaços desnecessários
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.strip())
    return df_pp



def removeEmojis(sentence):
    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def removeValores(sentence):
    new_sentece = ''

    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)

    return new_sentece

def usingStopwords(language):
    stop_words = []

    nltk.download('stopwords')

    if language == 'pt':
        stop_words = nltk.corpus.stopwords.words('portuguese')
    elif language == 'en':
        stop_words = nltk.corpus.stopwords.words('english')

    return stop_words

def removeStopwords(text, stop_words):
    tokens = []
    for word in text.split():
        if word not in stop_words:
            tokens.append(word)

    text = ' '.join(tokens)
    return text

def normalizarString(text):
    """
    Função para retirar acentuações e converter para minúscula
    :param text:
    :return text_normalizado
    """
    import unicodedata

    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text.lower())

## Load splits

In [17]:
train_raw = pd.read_csv('../assets/data/splits/train/raw.csv')
test_raw = pd.read_csv('../assets/data/splits/test/raw.csv')
df = pd.read_csv('../assets/data/df.csv')


In [18]:
df

,title,tags,link,label
0,Fed declara que relatório de empregos dos EUA ...,"['Desemprego', 'EUA', 'Fed', 'federal reserve']",https://www.sunoresearch.com.br/noticias/fed-e...,0
1,Petrobras não está na lista de privatizações p...,"['Economia', 'Eletrobras', 'Empresas', 'Jair B...",https://www.moneytimes.com.br/petrobras-nao-es...,1
2,"Destaques de Empresas: Oi (OIBR3), Petrobras (...","['destaques de empresas', 'Gol Linhas Aéreas (...",https://www.suno.com.br/noticias/destaques-de-...,1
3,Petrobras fecha venda de campos à Perenco por ...,"['Empresas', 'Petrobras', 'Petróleo', 'Reuters']",https://www.moneytimes.com.br/petrobras-fecha-...,1
4,Bancos partem para aquisições no digital; BTG ...,"['Itaú Unibanco (ITUB3/ITUB4)', 'Bradesco (BBD...",https://www.suno.com.br/noticias/bancos-partem...,3
...,...,...,...,...
18007,Tesouro Direto: Confira as variações dos títul...,['Tesouro Direto'],https://www.sunoresearch.com.br/noticias/tesou...,0
18008,Produção de petróleo e gás no país bate record...,"['ANP', 'Economia', 'Petrobras', 'Petróleo']",https://www.moneytimes.com.br/producao-de-petr...,1
18009,Petrobras inicia produção de quarta plataforma...,"['Empresas', 'Petrobras', 'Petróleo']",https://www.moneytimes.com.br/petrobras-inicia...,1
18010,Rodolfo Amstalden: Vale (VALE3) está menos ou ...,"['Colunista Empiricus', 'Colunistas', 'Comprar...",https://www.moneytimes.com.br/rodolfo-amstalde...,2


## Preprocessing splits

In [19]:
train_pp = transformDocument(train_raw, 'title', 'pt')
test_pp = transformDocument(test_raw, 'title', 'pt')
df_pp = transformDocument(df, 'title', 'pt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df_pp

,title,tags,link,label
0,fed declara relatorio empregos eua bom resultado,"['Desemprego', 'EUA', 'Fed', 'federal reserve']",https://www.sunoresearch.com.br/noticias/fed-e...,0
1,petrobras lista privatizacoes primeiro mandato...,"['Economia', 'Eletrobras', 'Empresas', 'Jair B...",https://www.moneytimes.com.br/petrobras-nao-es...,1
2,destaques empresas oi oibr3 petrobras petr4 sm...,"['destaques de empresas', 'Gol Linhas Aéreas (...",https://www.suno.com.br/noticias/destaques-de-...,1
3,petrobras fecha venda campos perenco total <NU...,"['Empresas', 'Petrobras', 'Petróleo', 'Reuters']",https://www.moneytimes.com.br/petrobras-fecha-...,1
4,bancos partem aquisicoes digital btg bpac11 de...,"['Itaú Unibanco (ITUB3/ITUB4)', 'Bradesco (BBD...",https://www.suno.com.br/noticias/bancos-partem...,3
...,...,...,...,...
18007,tesouro direto confira variacoes titulos nesta...,['Tesouro Direto'],https://www.sunoresearch.com.br/noticias/tesou...,0
18008,producao petroleo gas pais bate recorde maio d...,"['ANP', 'Economia', 'Petrobras', 'Petróleo']",https://www.moneytimes.com.br/producao-de-petr...,1
18009,petrobras inicia producao quarta plataforma ca...,"['Empresas', 'Petrobras', 'Petróleo']",https://www.moneytimes.com.br/petrobras-inicia...,1
18010,rodolfo amstalden vale vale3 menos atrativa,"['Colunista Empiricus', 'Colunistas', 'Comprar...",https://www.moneytimes.com.br/rodolfo-amstalde...,2


## Exporting preprocessed splits for EDA, word2vec and lstm_preprocessing

In [21]:
train_pp.to_csv('../assets/data/splits/train/preprocessed.csv', index=False)
test_pp.to_csv('../assets/data/splits/test/preprocessed.csv', index=False)
df_pp.to_csv('../assets/data/df_pp.csv', index=False)